<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT22/blob/hannahklingberg-Lab3/Lab3/hannahklingberg_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 3: Iterative Methods**
**Hanna Klingberg**

# **Abstract**

This lab explores fundamental methods to iteratively solve non-linear equations and equation systems. The functions defined in this lab are Jacobi iteration, Gauss-Seidel iteration, Newtons method for solving non-linear equations and Newtons method for solving non-linear equation systems. All functions and the theory behind them are described below. All defined algorithms passed their tests.

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [55]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [56]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np
import math

#try:
#    from dolfin import *; from mshr import *
#except ImportError as e:
#    !apt-get install -y -qq software-properties-common 
#    !add-apt-repository -y ppa:fenics-packages/fenics
#    !apt-get update -qq
#    !apt install -y --no-install-recommends fenics
#    from dolfin import *; from mshr import *
    
#import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

This lab explores iterative methods to solve non linear equations and equation systems. 



# **Method**

The theory behind the functions is described in the section of each function, as well as the method for testing the algorithms. 

# **Jacobi Iteration for Ax=b**

Jacobi Iteration iterates a solution x to Ax=b. The function takes A and b as input, and iterates according to the formula:
$x_i^{(k+1)} = a_{ii}^{-1}\Bigg( b_i - \sum_{j \neq i} a_{ij}x_j^{(k)} \Bigg) $              $\forall i$

The function uses the matrix D as the matrix with all diagonal elements of A, and D_inv as the inverse of A. The variable 'residual' is used as a stopping criteria for the iteration and it is defined as 1 initially and then redefined as the norm ||Ax-b|| after each iteration. The iteration continues until residual is smaller than $10^{-8}$, or until the number of max iterations is reached. The initial vector x is always defined as a zero vector. 

The tests are done by testing the residual norm ||Ax-b||, where x is the vector returned from the Jacobi iteration, and ||x-y||, where y is the manufactured solution to the equation Ax=b. 

(Reference: Chapter 7.7 of course book)

In [57]:
def jacobi_it(A,b):
  max_iter = 1000
  iter = 0
  n = A.shape[0]
  D = np.zeros((A.shape))
  for i in range(len(D)):
    D[i][i] = A[i][i]
  D_inv = np.linalg.inv(D)
  x1 = np.zeros(n)
  x2 = np.zeros(n)
  residual = 1
  while residual > 10**-8 and iter < max_iter:
    for i in range(n):
      sum = 0
      for j in range(n):
        if i != j:
          sum += A[i][j]*x1[j]
      x2[i] = D_inv[i][i] *(b[i]-sum)
    for i in range(n):
      x1[i] = x2[i]
    residual = np.linalg.norm(np.dot(A,x2)-b)
    iter += 1
  if np.isclose(np.linalg.norm(np.dot(A,x2)-b), 0, 10**-6):
    return x2
  else:
    return "did not converge"

In [58]:
def test_jacobi():
  passed = False
  A = np.array([[2,1],[1,2]])
  b = np.array([3,3])
  y = np.dot(np.linalg.inv(A), b)
  x = jacobi_it(A,b)
  test = np.subtract(x, y) 
  if not np.isclose(np.linalg.norm(test), 0, 10**-6):
    return False
  passed = True
  if not np.isclose(np.linalg.norm(np.dot(A,x) - b), 0, 10**-6):
    return False
  return passed


# **Gauss-Seidel iteration for Ax=b**

Gauss-Seidel iteration is an iterative method to find the solution x for Ax = b, where x is iteratively calculated as:

$x_i^{(k+1)} = a_{ii}^{-1} \Bigg( b_i - \sum_{j < i} a_{ij} x_j^{(k+1)} - \sum_{j > i} a_{ij}x_j^{(k)} \Bigg)$ $\forall i $

The function takes a matrix A and vector b as input. It assumes the initial x as a zero vector, and then iteratively calculates x in the while loop. The stopping criteria are the same as for the Jacobi iteration. The initial vector x is always defined as a zero vector. 

The function is tested by checking the norms ||x-y||, where y is the manufactured solution to Ax=b, and ||Ax-b|| for the vector x returned by the function. 

(Reference: Chapter 7.7 of course book)

In [59]:
def gauss_seidel(A,b):
  max_iter = 1000
  iter = 0
  n = A.shape[0]
  x = np.zeros(n)
  residual = 1
  while residual > 10**-8 and iter < max_iter:
    for i in range(n):
      sum1 = 0
      sum2 = 0 
      for j in range(i):
          sum1 += A[i][j]*x[j]
      for j in range(i+1, n):
        sum2 += A[i][j]*x[j] 
      x[i] =(1/A[i,i])*(b[i]-sum1 - sum2)
    residual = np.linalg.norm(np.dot(A,x)-b)
    iter += 1
  if np.isclose(np.linalg.norm(np.dot(A,x)-b), 0, 10**-6):
    return x
  else:
    return "did not converge" 

In [60]:
def test_gauss_seidel():
  passed = False
  A = np.array([[2,1],[1,2]])
  b = np.array([3,3])
  y = np.dot(np.linalg.inv(A), b)
  x = gauss_seidel(A,b)
  test = np.subtract(x, y) 
  if not np.isclose(np.linalg.norm(test), 0, 10**-6):
    return False
  passed = True
  if not np.isclose(np.linalg.norm(np.dot(A,x) - b), 0, 10**-6):
    return False
  return passed


# **Newtons Method for the Non-linear Equation**

Newtons method is an iterative method for solving non linear equation $f(x) = 0$. The method can be interpreted as that $x^{(k+1)}$ is determined by the tangent line of the function $f(x)$ at $x^{(k)}$

The algorithm takes the function and its derivative as input, and itertatively approximates x. The initial guess $x_0$ is always 1. The iteration ends when either the absolute vale of f(x) is less than $10^{-8}$ or when the maximum number of iterations is reached. The solution is checked before it's returned, and it is determined whether it is a correct solution or if the function did not converge. 

The testing is done by checking that |f(x)| = 0 and |x-y| = 0 where x is the solution returned by the algorithm and y is the manufactured solution to the algorithm. 

(Reference: Chapter 8.3 of course book)

In [61]:

def newtons_method(f, derivative):
  x = 1
  TOL = 10**-8
  iter = 0
  max_iter = 1000
  while abs(f(x)) > TOL and iter < max_iter:
    fx = f(x)
    derivativex = derivative(x)
    if derivativex == 0:
      return "Error, derivative is 0"
    x -= fx / derivativex
    iter +=1
  if np.isclose(f(x), 0, 10**-6):
    return x 
  else:
    return "did not converge"

In [62]:
def test_newton():
  passed = False
  test1 = newtons_method(lambda x: math.cos(x), lambda x: -math.sin(x))
  if np.isclose(math.cos(test1), 0, 10**-6):
    passed = True
  else:
     return False
  if np.isclose((math.pi/2) - test1, 0, 10**-6):
    passed = True
  else:
    return False
  func2 = lambda x: x**2 - 3
  deriv2 = lambda x: 2*x 
  test2 = newtons_method(func2, deriv2)
  if np.isclose(func2(test2), 0, 10**-6):
    passed = True
  else:
    return False
  if np.isclose(math.sqrt(3) - test2, 0, 10**-6):
    passed = True
  else:
    return False
  return passed


# **Newtons Method for vector nonlinear equation**
Newtons method for vector nonlinear equation is an iterative method for solving a system of non-linear equations. The algorithm is implemented according to the pseudo-code given in algorithm 8.4 (chapter 8.4, course book). 

The algorithm takes as input a function f, its jacobian matrix df and n, which is the length of the output vector x. It returns the estimated solution x such that f(x) = 0. 

The testing is done by calculating ||f(x)|| and ||x-y||, where x is the returned solution and y is a manufactured solution. The function and matrix used as test input is taken from a former students work (https://github.com/johanhoffman/DD2363-VT20/blob/master/Lab-3/iZafiro_lab3.ipynb )

(Reference: Chapter 8.4 of course book)


In [63]:
def newtons_vec_met(f, df, n):
  x = np.zeros(n)
  residual = 1
  while residual > 10**-8:
    fx = f(x)
    dfx = df(x)
    if np.linalg.det(dfx) == 0:
      return "determinant of jacobian is 0, so it is not invertible"
    dx = - np.dot(np.linalg.inv(dfx), fx)
    x += dx
    residual = np.linalg.norm(fx)
  if np.isclose(np.linalg.norm(f(x)), 0, 10**-6):
    return x 
  else:
    return "did not converge"

In [64]:
def test_newton_vec():
  passed = False
  func1 = lambda x: np.array([2*x[0] - 3*x[1] + 5, 4*x[0] - 7*x[1] + 10])
  jacob1 = lambda x: np.array([[2, -3], [4, -7]])
  test = newtons_vec_met(func1, jacob1, 2)
  norm = np.linalg.norm(func1(test))
  if not np.isclose(norm, 0, 10**-8):
    return False
  passed = True
  sol = [-2.5, 0]
  if not np.isclose(np.linalg.norm(test - sol), 0, 10**-8):
    return False
  return passed



# **Results**

All functions passed their tests, as can be seen below. The test conditions are defined in the definition of each function. 

In [65]:
def test_all():
  passed = 0
  if test_jacobi():
    passed +=1
    print("Jacobi Iteration passed the tests")
  if test_gauss_seidel():
    passed +=1
    print("Gauss Seidel Iteration passed the tests")
  if test_newton():
    passed +=1
    print("Newtons Method passed the tests")
  if test_newton_vec():
    passed += 1
    print("Newtons Vector Method passed the tests")
  print("%d of 4 functions passed their tests" %(passed))
test_all()
  

Jacobi Iteration passed the tests
Gauss Seidel Iteration passed the tests
Newtons Method passed the tests
Newtons Vector Method passed the tests
4 of 4 functions passed their tests


# **Discussion**

All functions passed their tests, and are thus correctly implemented. I have not implemented any error handling, and thus assume that the input is correct, which is a weakness of the algorithms. The two first iteration functions, Jacobi and Gauss-Seidel, could also be solved with the use of Richardson iteration, but I chose not to do so and instead use the equations as defined in the book. For the Newtons method to solve non-linear equation systems I had some trouble to come up with a test case, so I used one from a former student. 